In [1]:
#!/usr/bin/env python

"""
Threaded Downloader
"""
import time
import requests
import threading
import queue


def main():

    NEWS_API_KEY = "1644ea0ae3684a9aa16b0cffed5bbdcc"

    base_url = 'https://newsapi.org/v1/'
    WORD = "war"
    TITLES = []    
    thread_count = 4
    art_count = 0
    word_count = 0
    
    def get_sources():
        url = base_url + "sources"
        params = {"language":"en"}
        resp = requests.get(url, params=params)
        data = resp.json()
        sources = [src['id'].strip() for src in data['sources']]
        print("all the sources")
        print(sources)
        return sources

    def get_articles(sources):
        for source in sources:
            url = base_url + "articles"
            params = {"source": source,
                      "apiKey": NEWS_API_KEY,
                      "sortBy": "top",
                      }
            print("requesting:", source)
            resp = requests.get(url, params=params)
            if resp.status_code != 200:  # aiohttpp has "status"
                print("something went wrong with {}".format(source))
                print(resp)
                print(resp.text)
                return []
            data = resp.json()
            for art in data['articles']:
                TITLES.append(str(art['title']) + str(art['description']))

    def articles_threading(sources, thread_count=1):
        results = queue.Queue()
        sources = sources
        threads = []

        def worker(*args):
            results.put(get_articles(*args))

        for i in range(thread_count):
            thread = threading.Thread(target=worker, args=(sources,))
            threads.append(thread)
            thread.start()
            print("Thread %s started" % thread.name)

        for t in threads:
            t.join()

    def count_word(word, titles):
        word = word.lower()
        count = 0
        for title in titles:
            if word in title.lower():
                count += 1
        return count

    start = time.time()
    sources = get_sources()
    articles_threading(sources, thread_count)
    count_word(WORD, TITLES)
    art_count += len(TITLES)
    word_count += count_word(WORD, TITLES)

    print(WORD, "found {} times in {} articles".format(word_count, art_count))
    print("Process took {:.0f} seconds".format(time.time() - start))


if __name__ == '__main__':
    main()

all the sources
['abc-news-au', 'al-jazeera-english', 'ars-technica', 'associated-press', 'bbc-news', 'bbc-sport', 'bloomberg', 'breitbart-news', 'business-insider', 'business-insider-uk', 'buzzfeed', 'cnbc', 'cnn', 'daily-mail', 'engadget', 'entertainment-weekly', 'espn', 'espn-cric-info', 'financial-times', 'football-italia', 'fortune', 'four-four-two', 'fox-sports', 'google-news', 'hacker-news', 'ign', 'independent', 'mashable', 'metro', 'mirror', 'mtv-news', 'mtv-news-uk', 'national-geographic', 'new-scientist', 'newsweek', 'new-york-magazine', 'nfl-news', 'polygon', 'recode', 'reddit-r-all', 'reuters', 'talksport', 'techcrunch', 'techradar', 'the-economist', 'the-hindu', 'the-huffington-post', 'the-lad-bible', 'the-new-york-times', 'the-next-web', 'the-sport-bible', 'the-telegraph', 'the-times-of-india', 'the-verge', 'the-wall-street-journal', 'the-washington-post', 'time', 'usa-today']
requesting:Thread Thread-4 started abc-news-au

requesting: Thread Thread-5 started
requesting: